In [24]:
from mace.calculators import mace_mp
from ase import build

from ase.md import Langevin
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase import units
from ase import io
from ase.optimize import BFGS, LBFGS, FIRE, LBFGSLineSearch
from ase.visualize import view
from ase.io import Trajectory
from ase import Atoms
from ase.build import bulk
import numpy as np
import matplotlib.pyplot as plt

macemp = mace_mp(default_dtype="float64")
macemp_d = mace_mp(dispersion=True, default_dtype="float64")
macemp_omat = mace_mp(model="/Users/joehart/Desktop/0_Cambridge/0_MPhil_Scientific_Computing/Written_assignments/MACE-MP-0/Notebooks_mace/mace_test/mace-omat-0-medium.model", dispersion=True, default_dtype="float64")


Using medium MPA-0 model as default MACE-MP model, to use previous (before 3.10) default model please specify 'medium' as model argument
Using Materials Project MACE for MACECalculator with /Users/joehart/.cache/mace/macempa0mediummodel
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using medium MPA-0 model as default MACE-MP model, to use previous (before 3.10) default model please specify 'medium' as model argument
Using Materials Project MACE for MACECalculator with /Users/joehart/.cache/mace/macempa0mediummodel
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.


/Users/joehart/Desktop/0_Cambridge/0_MPhil_Scientific_Computing/Written_assignments/MACE-MP-0/Notebooks_mace/mace_test/venv_mace/lib/python3.12/site-packages/mace/calculators/mace.py:139: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)
/Users/joehart/Desktop/0_Cambridge/0_MPhil_Scientific_Computing/Written_assignments/MACE-MP-0/Notebooks_mace/mace_test/venv_mace/lib/python3.12/site-packages/mace/calculators/mace.py:139: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using TorchDFTD3Calculator for D3 dispersion corrections
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using TorchDFTD3Calculator for D3 dispersion corrections


/Users/joehart/Desktop/0_Cambridge/0_MPhil_Scientific_Computing/Written_assignments/MACE-MP-0/Notebooks_mace/mace_test/venv_mace/lib/python3.12/site-packages/mace/calculators/mace.py:139: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


In [2]:
import importlib
import interlayer_spacing
importlib.reload(interlayer_spacing)

from interlayer_spacing import calculate_interlayer_spacing, set_interlayer_spacing

In [17]:
mxene_relaxed = io.read("relaxed_mxene.xyz")
graphene_atoms = io.read("graphene_atoms.xyz")
go_relax = io.read("go_relax.xyz")
goh_relax = io.read("goh_relax.xyz")

In [18]:
graphene_atoms.get_tags()

array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
      dtype=int32)

In [19]:
def create_heterostructure(mxene, graphene_layer, height):
    # Stack MXene and graphene-based layer with a given interlayer spacing (height)
    graphene_layer = graphene_layer.copy()
    
    mxene_avg_top_O_z = np.mean([atom.position[2] for atom in mxene if atom.tag == 3])

    #mxene_avg_top_C_z = np.mean([atom.position[2] for atom in mxene if atom.symbol == 'C'])
    #mxene_avg_top_O_z = np.mean([atom.position[2] for atom in mxene if atom.symbol == 'O' and atom.position[2] > mxene_avg_top_C_z])
    
    graphene_layer.positions[:, 2] += mxene_avg_top_O_z - np.average(graphene_layer.positions[:, 2]) + height
    heterostructure = mxene + graphene_layer
    print("MXene avg top O z: ", mxene_avg_top_O_z)

    return heterostructure


m_g_2 = create_heterostructure(mxene_relaxed, graphene_atoms, 2)

MXene avg top O z:  12.23159545


In [12]:
m_g_2_relaxed = m_g_2.copy()
m_g_2_relaxed.calc = macemp
optimiser = BFGS(m_g_2_relaxed)
optimiser.run(fmax=0.001, steps=5000)

      Step     Time          Energy          fmax
BFGS:    0 11:57:08    -1148.442193        9.571269
BFGS:    1 11:57:13    -1152.298626        6.308827
BFGS:    2 11:57:17    -1155.487765        2.398838
BFGS:    3 11:57:21    -1157.619832        1.581759
BFGS:    4 11:57:29    -1159.302757        2.264770
BFGS:    5 11:57:32    -1161.116164        2.157342
BFGS:    6 11:57:37    -1162.607413        1.611678
BFGS:    7 11:57:41    -1164.864055        1.475116
BFGS:    8 11:57:44    -1166.747696        2.451678
BFGS:    9 11:57:48    -1168.617591        2.238572
BFGS:   10 11:57:51    -1170.159747        0.950911
BFGS:   11 11:57:58    -1171.154016        1.339281
BFGS:   12 11:58:06    -1172.645074        1.642200
BFGS:   13 11:58:10    -1173.701872        1.709449
BFGS:   14 11:58:15    -1174.617134        1.171659
BFGS:   15 11:58:19    -1175.202747        0.589129
BFGS:   16 11:58:22    -1175.507827        0.523086
BFGS:   17 11:58:26    -1175.758027        0.715945
BFGS:   18 11:

True

In [13]:
view(m_g_2_relaxed, viewer='x3d')

In [15]:
calculate_interlayer_spacing(m_g_2_relaxed)

/Users/joehart/Desktop/0_Cambridge/0_MPhil_Scientific_Computing/Written_assignments/MACE-MP-0/Notebooks_mace/mace_test/venv_mace/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/joehart/Desktop/0_Cambridge/0_MPhil_Scientific_Computing/Written_assignments/MACE-MP-0/Notebooks_mace/mace_test/venv_mace/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


nan

In [22]:
from ase.md import Bussi
def printenergy(a=m_g_2):
    """Function to print the potential, kinetic and total energy"""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))
    
    
m_g_2.calc = macemp
T_init = 300
MaxwellBoltzmannDistribution(m_g_2, temperature_K=T_init)


dyn = Bussi(m_g_2, 0.5 * units.fs, T_init, 0.01)
n_steps = 1000

dyn.attach(printenergy, interval = 100) 

traj = Trajectory('mg_2_test.traj', 'w', m_g_2)
dyn.attach(traj.write, interval = 50)
printenergy()
dyn.run(n_steps)

Energy per atom: Epot = -8.834eV  Ekin = 0.041eV (T=321K)  Etot = -8.793eV
Energy per atom: Epot = -8.834eV  Ekin = 0.041eV (T=321K)  Etot = -8.793eV
Energy per atom: Epot = -8.975eV  Ekin = 0.038eV (T=297K)  Etot = -8.936eV
Energy per atom: Epot = -9.017eV  Ekin = 0.050eV (T=389K)  Etot = -8.966eV
Energy per atom: Epot = -9.040eV  Ekin = 0.041eV (T=318K)  Etot = -8.999eV
Energy per atom: Epot = -9.047eV  Ekin = 0.046eV (T=354K)  Etot = -9.001eV
Energy per atom: Epot = -9.049eV  Ekin = 0.038eV (T=295K)  Etot = -9.010eV
Energy per atom: Epot = -9.041eV  Ekin = 0.041eV (T=314K)  Etot = -9.001eV
Energy per atom: Epot = -9.041eV  Ekin = 0.042eV (T=326K)  Etot = -8.998eV
Energy per atom: Epot = -9.041eV  Ekin = 0.038eV (T=290K)  Etot = -9.003eV
Energy per atom: Epot = -9.041eV  Ekin = 0.039eV (T=299K)  Etot = -9.003eV
Energy per atom: Epot = -9.041eV  Ekin = 0.039eV (T=301K)  Etot = -9.002eV


True

In [23]:
from ase.md import Bussi
def printenergy(a=m_g_2):
    """Function to print the potential, kinetic and total energy"""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))
    
    
m_g_2.calc = macemp_d
T_init = 300
MaxwellBoltzmannDistribution(m_g_2, temperature_K=T_init)


dyn = Bussi(m_g_2, 0.5 * units.fs, T_init, 0.01)
n_steps = 1000

dyn.attach(printenergy, interval = 100) 

traj = Trajectory('mg_2_test_d.traj', 'w', m_g_2)
dyn.attach(traj.write, interval = 50)
printenergy()
dyn.run(n_steps)

/Users/joehart/Desktop/0_Cambridge/0_MPhil_Scientific_Computing/Written_assignments/MACE-MP-0/Notebooks_mace/mace_test/venv_mace/lib/python3.12/site-packages/torch_dftd/torch_dftd3_calculator.py:98: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  cell: Optional[Tensor] = torch.tensor(


Energy per atom: Epot = -9.217eV  Ekin = 0.039eV (T=303K)  Etot = -9.178eV
Energy per atom: Epot = -9.217eV  Ekin = 0.039eV (T=303K)  Etot = -9.178eV
Energy per atom: Epot = -9.180eV  Ekin = 0.041eV (T=320K)  Etot = -9.139eV
Energy per atom: Epot = -9.186eV  Ekin = 0.034eV (T=262K)  Etot = -9.152eV
Energy per atom: Epot = -9.184eV  Ekin = 0.033eV (T=257K)  Etot = -9.151eV
Energy per atom: Epot = -9.185eV  Ekin = 0.037eV (T=289K)  Etot = -9.148eV
Energy per atom: Epot = -9.182eV  Ekin = 0.041eV (T=317K)  Etot = -9.141eV
Energy per atom: Epot = -9.186eV  Ekin = 0.040eV (T=309K)  Etot = -9.146eV
Energy per atom: Epot = -9.187eV  Ekin = 0.040eV (T=310K)  Etot = -9.147eV
Energy per atom: Epot = -9.192eV  Ekin = 0.039eV (T=299K)  Etot = -9.154eV
Energy per atom: Epot = -9.186eV  Ekin = 0.040eV (T=310K)  Etot = -9.146eV
Energy per atom: Epot = -9.180eV  Ekin = 0.034eV (T=266K)  Etot = -9.146eV


True